In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/spotify-data/StreamingHistory0.json
/kaggle/input/spotify-data/Playlist1.json
/kaggle/input/spotify-data/YourLibraryUpdated.json
/kaggle/input/spotify-data/StreamingHistory1.json


Import the required libraries

In [2]:
import numpy as np
import pandas as pd
import requests

First step, reading the streaming history files and then merging them together to obtain a final data frame

In [3]:
df_stream0 = pd.read_json('/kaggle/input/spotify-data/StreamingHistory0.json')
df_stream1 = pd.read_json('/kaggle/input/spotify-data/StreamingHistory1.json')

#merge
df_stream = pd.concat([df_stream0, df_stream1])

As many tracks might have the same name let's create an Unique ID for every track by combining artist and track name

In [4]:
df_stream['Unique_ID'] = df_stream['artistName']+":"+df_stream['trackName']
df_stream.head()

,endTime,artistName,trackName,msPlayed,Unique_ID
0,2022-05-07 18:19,Taylor Swift,This Love,98020,Taylor Swift:This Love
1,2022-05-07 20:16,Gryffin,Bye Bye (feat. Ivy Adara),400,Gryffin:Bye Bye (feat. Ivy Adara)
2,2022-05-08 16:16,Selena Gomez,Dance Again,5200,Selena Gomez:Dance Again
3,2022-05-08 16:23,Julia Michaels,Anxiety (with Selena Gomez),267490,Julia Michaels:Anxiety (with Selena Gomez)
4,2022-05-08 16:23,Selena Gomez,Back To You,1990,Selena Gomez:Back To You


A quick note: Please remember to clean your YourLibrary.json file so that it only contains the tracks row bounded by square brackets []. Otherwise it will throw an error saying the rows must be of equal length for it to be converted to a data frame.

In [5]:
# read your edited Library json file into a pandas dataframe
df_library = pd.read_json('/kaggle/input/spotify-data/YourLibraryUpdated.json')

# add UniqueID column (same as above)
df_library['Unique_ID'] = df_library['artist'] + ":" + df_library['track']

# add column with track URI stripped of 'spotify:track:'
new = df_library["uri"].str.split(":", expand = True)
df_library['track_uri'] = new[2]

df_library.head()


,artist,album,track,uri,Unique_ID,track_uri
0,Lorde,Solar Power,Solar Power,spotify:track:3KdoeNlEN0BoAKWzaRLNZa,Lorde:Solar Power,3KdoeNlEN0BoAKWzaRLNZa
1,Liam Payne,LP1,Remember,spotify:track:300IKIOz64Pji16h7hUSRU,Liam Payne:Remember,300IKIOz64Pji16h7hUSRU
2,Billie Eilish,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",when the party's over,spotify:track:43zdsphuZLzwA9k4DJhU0I,Billie Eilish:when the party's over,43zdsphuZLzwA9k4DJhU0I
3,Jeremy Zucker,brent ii,this is how you fall in love,spotify:track:1aEsTgCsv8nOjEgyEoRCpS,Jeremy Zucker:this is how you fall in love,1aEsTgCsv8nOjEgyEoRCpS
4,Harry Styles,Harry's House,Grapejuice,spotify:track:4j6kId9QIqhoXvqHKgSKa0,Harry Styles:Grapejuice,4j6kId9QIqhoXvqHKgSKa0


Now by using the YourLibraryUpdated data frame let's find out tracks from our streaming history which are added to our library

In [6]:
df_final = df_stream.copy()

#to check whether a track exists in my library or not
df_final['In Library'] = np.where(df_final['Unique_ID'].isin(df_library['Unique_ID'].tolist()), 1, 0)

#now let's left join with stream data to check whether the track is present in library or not
df_final = pd.merge(df_final, df_library[['album', 'Unique_ID', 'track_uri']], how = 'left', on = ['Unique_ID'])

In [7]:
df_final

,endTime,artistName,trackName,msPlayed,Unique_ID,In Library,album,track_uri
0,2022-05-07 18:19,Taylor Swift,This Love,98020,Taylor Swift:This Love,0,NaN,NaN
1,2022-05-07 20:16,Gryffin,Bye Bye (feat. Ivy Adara),400,Gryffin:Bye Bye (feat. Ivy Adara),0,NaN,NaN
2,2022-05-08 16:16,Selena Gomez,Dance Again,5200,Selena Gomez:Dance Again,1,Rare,5sK3o66yupTNIK6gWgzGjf
3,2022-05-08 16:23,Julia Michaels,Anxiety (with Selena Gomez),267490,Julia Michaels:Anxiety (with Selena Gomez),0,NaN,NaN
4,2022-05-08 16:23,Selena Gomez,Back To You,1990,Selena Gomez:Back To You,0,NaN,NaN
...,...,...,...,...,...,...,...,...
12195,2023-05-08 06:41,Harry Styles,Cinema,93884,Harry Styles:Cinema,1,Harry's House,35TyJIMR3xRouUuo2sjS6v
12196,2023-05-08 06:43,Harry Styles,Daydreaming,151440,Harry Styles:Daydreaming,1,Harry's House,69w5X6uTrOaWM32IetSzvO
12197,2023-05-08 06:50,Harry Styles,Keep Driving,192268,Harry Styles:Keep Driving,1,Harry's House,2NcQic8JxdjAlAHuNbOIRE
12198,2023-05-08 06:56,Harry Styles,Satellite,328304,Harry Styles:Satellite,1,Harry's House,0rzaRSujxA0bKyjJl6vHYq


Yay! All the data cleaning and processing is done. Now it's time for the exciting part - working with the Spotify API

In [8]:
CLIENT_ID = 'your-client-id'
CLIENT_SECRET = 'your-client-secret'

Note: This access token needs to be regenerated after an hour

In [9]:
# generate access token

# authentication URL
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [10]:
# used for authenticating all API calls
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [11]:
# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

All API authentication requirements are now set up. Let's pull data using the API like artists and genres from track uri

In [12]:
#create a blank dictionary
dict_genre = {}

# convert track_uri column to an iterable list
track_uris = df_library['track_uri'].to_list()

# loop through track URIs and pull artist URI using the API,
# then use artist URI to pull genres associated with that artist
# store all these in a dictionary
for track in track_uris:
    dict_genre[track] = {'artist_uri': "", "genres":[]}
    
    r = requests.get(BASE_URL + 'tracks/' + track, headers=headers)
    r = r.json()
    
    a_uri = r['artists'][0]['uri'].split(':')[2]
    dict_genre[track]['artist_uri'] = a_uri
    
    s = requests.get(BASE_URL + 'artists/' + a_uri, headers=headers)
    s = s.json()
    dict_genre[track]['genres'] = s['genres']

In [13]:
#converting dictionary to dataframe
df_genre = pd.DataFrame.from_dict(dict_genre, orient='index')
df_genre.insert(0, 'track_uri', df_genre.index)
df_genre.reset_index(inplace=True, drop=True)

df_genre.head()

,track_uri,artist_uri,genres
0,3KdoeNlEN0BoAKWzaRLNZa,163tK9Wjr9P9DmM0AVK7lm,"[art pop, metropopolis, nz pop, pop]"
1,300IKIOz64Pji16h7hUSRU,5pUo3fmmHT8bhCyHE52hA6,[pop]
2,43zdsphuZLzwA9k4DJhU0I,6qqNVTkY8uBg9cP3Jd7DAH,"[art pop, electropop, pop]"
3,1aEsTgCsv8nOjEgyEoRCpS,3gIRvgZssIb9aiirIg0nI3,"[alt z, electropop, pop]"
4,4j6kId9QIqhoXvqHKgSKa0,6KImCVD70vtIoJWnq6nGn3,[pop]


In [14]:
df_genre_expand = df_genre.explode('genres')
df_genre_expand

,track_uri,artist_uri,genres
0,3KdoeNlEN0BoAKWzaRLNZa,163tK9Wjr9P9DmM0AVK7lm,art pop
0,3KdoeNlEN0BoAKWzaRLNZa,163tK9Wjr9P9DmM0AVK7lm,metropopolis
0,3KdoeNlEN0BoAKWzaRLNZa,163tK9Wjr9P9DmM0AVK7lm,nz pop
0,3KdoeNlEN0BoAKWzaRLNZa,163tK9Wjr9P9DmM0AVK7lm,pop
1,300IKIOz64Pji16h7hUSRU,5pUo3fmmHT8bhCyHE52hA6,pop
...,...,...,...
1031,2P4OICZRVAQcYAV2JReRfj,1vCWHaC5f2uS3yhpwWbIA6,edm
1031,2P4OICZRVAQcYAV2JReRfj,1vCWHaC5f2uS3yhpwWbIA6,pop
1031,2P4OICZRVAQcYAV2JReRfj,1vCWHaC5f2uS3yhpwWbIA6,pop dance
1032,2bZMOs3RjmhhGca6MEzjyl,6tbjWDEIzxoDsBA1FuhfPW,dance pop


Lastly, save df_final and df_genre_expand as csv files in order to work with them in Tableau

In [15]:
df_final.to_csv('MySpotifyData.csv')
df_genre_expand.to_csv('GenresExpanded.csv')